In [ ]:
# 设置主要贡献方面
ITEMs_main = ['Pigmeat', 'Fish, Seafood', 'Eggs']

In [ ]:
def mkPVs_compareAreaToGroup(
    domain,
    area,
    element,
    items_group='',
    items=[],
    group='World',
):
    # set items
    if items_group=='' : 
        if items==[]: items=ITEMs.copy()
        else: items=items.copy()
        
    else:
        items_raw = CDTs[domain][area][element][items_group]
        items=[]
        for i in items_raw:
            if i in ITEMs_all:items.append(i)
    
    # transposed from PVs (Area-Year) to new pivots (Year-Area) for a particular item
    pivots={}
    n=0
    for i in items:
        n+=1
        pivots[i]=PVs_AreaYear[domain][area][element][i].copy()
        pivots[i]=pivots[i][pivots[i].index.isin([area, group])].transpose()
        pivots[i].index.names = ['Year']
#         print(i)
#         display(pivots[i].tail(2))
    
    # make a pivot for a particular area where 'Area' as index, items as columns
    pivot_A=pd.DataFrame()
    for i in items:
        pivot_A = pd.concat([pivot_A, pivots[i][area]], axis=1)
    pivot_A.columns=items
#     print(area)
#     display(pivot_A.tail(2))
    
    # make a pivot for the group where the area is in, where 'Area' as index, items as columns
    pivot_G=pd.DataFrame()
    for i in items:
        pivot_G = pd.concat([pivot_G, pivots[i][group]], axis=1)
    pivot_G.columns=items
#     print(group)
#     display(pivot_G.tail(2))
    
    # a dictionary contains 3 pivots above
    pivots_AG={
        (area,group):pivots,
        area:pivot_A,
        group:pivot_G,
    }
    return pivots_AG

In [ ]:
def proportion_AreaToGroup(
    domain,
    area,
    element,
    items_group='',
    items=[],
    group='World',
    toUnit='',
    changeUnit=1,
    kind='area',
):        
    # set items
    if items_group=='' : 
        if items==[]: items=ITEMs.copy()
        else: items=items.copy()
        
    else:
        items_raw = CDTs[domain][area][element][items_group]
        items=[]
        for i in items_raw:
            if i in ITEMs_all:items.append(i)
    
    # make a dictionary of 'mkPVs_compareAreaToGroup()'
    PVs_AG[domain]={}
    PVs_AG[domain][area]={}
    PVs_AG[domain][area][element]=mkPVs_compareAreaToGroup(domain, area, element,items=items)

    # plot settings
    fig_row=(len(items)+1)//2
    fig_size=(10,fig_row*3)
    
    title = "{}: {} VS {}".format(element, area, group)
    
    if kind=='area':alpha=0.4
    else:alpha=1
        
    X_step=10
    X_ticks=range(Years[-1]-50,Years[-1]+X_step,X_step)
    
    if toUnit=='': Y_label = element
    else: 
        if '(' in element: Y_label = element[:element.find('(')] + '({})'.format(toUnit)
        else: Y_label=element+' ({})'.format(toUnit)

    # ===
    fig = plt.figure(figsize = fig_size,dpi=300)
    fig.tight_layout()                          
    plt.subplots_adjust(wspace =0.3, hspace =0.3)
    plt.suptitle(title, fontsize=TIT_size)
    
    n=0
    for i in items:
        n+=1
        ax = fig.add_subplot(fig_row,2,n)
        plt.title(i, fontsize=TIT_size)
        df = PVs_AG[domain][area][element][(area,group)][i].copy()
        df=df*changeUnit
        df.plot(kind=kind,stacked=False, alpha=alpha, ax=plt.gca())

        ax=plt.gca()
        ax.legend(fontsize=LEG_size, loc='upper left')

        X_step=5
        setX_quant(ax, X_ticks, X_step)

        Y_max=df.values.max()
        setY_quant(ax, 1.5*Y_max, Y_label)
    
#     path = './results/FAO_LivestockFish_FoodSupplyQuantity/img/Fig_4_'  + title[:title.find('\/')] + '.png'
#     bbox = matplotlib.transforms.Bbox([[-1, -1], list(map(lambda x:x+2, list(fig_size)))])
#     fig.savefig(path, format='png', transparent=False, dpi=300, bbox_inches =bbox)
#     print('Img saved as: ' + path)

In [ ]:
proportion_AreaToGroup(D,A,E, items=items, toUnit='10,000 t', changeUnit=1/10000, kind='area')

In [ ]:
def pi_compareAreaToGroup(
    domain,
    area,
    element,
    items_group='',
    items=[],
    years=[Years[0],Years[-1]],
    group='World',
    other='Other Areas',
    colors=CLs,
    fig_size=(13,6),
):
    
    # set items
    if items_group=='' : 
        if items==[]: items=ITEMs.copy()
        else: items=items.copy()
    else:
        items_raw = CDTs[domain][area][element][items_group]
        items=[]
        for i in items_raw:
            if i in ITEMs_all:items.append(i)
                
    # plot settings
    title = "Proportions of Different {} in {} and the Proportions {} takes".format(element, group, area)
        
    # plot all items in the world...
    
    fig = plt.figure(figsize = fig_size,dpi=300)
    fig.tight_layout()                          
    plt.subplots_adjust(wspace =0.3, hspace =0.2)
    plt.suptitle(title, fontsize=TIT_size)

    fig_no=0
    for y in years:
        fig_no+=1

        df=pd.concat([PVs_AG[domain][area][element][area].loc[y],
                      PVs_AG[domain][area][element][group].loc[y]],axis=1).copy()

        df=df[df.index.isin(items)]
        df.columns=[area,group]
        df[other] = df[group] - df[area]

        print(y)
        display(df)

        ax = fig.add_subplot(1,2, fig_no)
        plt.title(y, fontsize=TIT_size)

        size = 0.5
        outer_radius = 2
        inner_colors = ['r', 'lightgray', 'r', 'lightgray', 'r', 'lightgray', 'r', 'lightgray']

        outer_explode = np.zeros(len(items))
        inner_explode = np.zeros(len(items*2))

        for n in range(len(items)):
            if items[n] in ITEMs_main:
                outer_explode[n] = 0.15
                inner_explode[2*n] = 0.15

        area_pct = ['{}{}'.format(int(df.loc[i][area]/df.loc[i][group]*100),'%') for i in df.index.values]
        inner_labels = []
        for i in area_pct:
            inner_labels.append(i)
            inner_labels.append('')

        shadow = False
        startangle = 0
        plt.pie(df[group].values, 
                autopct='%1.0f%%',
                pctdistance=1.06,
                explode=list(outer_explode),
                radius=outer_radius, 
                colors=colors,
                center = (0,0),
                textprops = {'fontsize':LEG_size}, 
                wedgeprops=dict(width=size),
                shadow=shadow,
                startangle=startangle,
                )
        plt.pie(df[[area,other]].values.flatten(), 
                labels=inner_labels,
                labeldistance = 0.8,
                explode=list(inner_explode),
                radius=outer_radius-size, 
                colors=inner_colors,
                center = (0,0),
                textprops = {'fontsize':LEG_size}, 
                wedgeprops=dict(width=size),
                shadow=shadow,
                startangle=startangle,
               )
        plt.axis('equal')          # equal 使得为正圆

    ax=plt.gca()
    ax.legend(labels=list(df.index.values)+[area,other],frameon=False,fontsize=LEG_size, 
              loc='lower center',bbox_to_anchor=(-0.1,-0.1),borderaxespad = 0.)
    
#     path = './results/FAO_LivestockFish_FoodSupplyQuantity/img/Fig_5_'  + title[:title.find('\/')] + '.png'
#     bbox = matplotlib.transforms.Bbox([[-1, -1], list(map(lambda x:x+2, list(fig_size)))])
#     fig.savefig(path, format='png', transparent=False, dpi=300, bbox_inches =bbox)
#     print('Img saved as: ' + path)

In [ ]:
pi_compareAreaToGroup(D,A,E)